In [130]:
import json
import os
import re
import pandas as pd

In [131]:
results_path = r"Yi-34B-Chat-Yi-34B-Chat-interview-results-02182024"
full_path = os.path.join(results_path)
full_path

'Yi-34B-Chat-Yi-34B-Chat-interview-results-02182024'

In [132]:
interview_path = os.path.join(r"Yi-34B-Chat-Yi-34B-Chat-interview-results-02182024\landlord-Yi-34B-Chat-Farhad Abbasi from Iran-Yi-34B-Chat-München-20240207\1b9338bb-5c66-4af1-9210-0f7fef33ec72\renter", "conversation.json")
print(interview_path)
conversation_history = json.load(open(interview_path))

Yi-34B-Chat-Yi-34B-Chat-interview-results-02182024\landlord-Yi-34B-Chat-Farhad Abbasi from Iran-Yi-34B-Chat-München-20240207\1b9338bb-5c66-4af1-9210-0f7fef33ec72\renter\conversation.json


In [133]:
conversation_history

[{'content': 'What is your name?', 'role': 'user', 'name': 'Evaluator'},
 {'content': 'My name is Farhad Abbasi from Iran.',
  'role': 'user',
  'name': 'Farhad Abbasi'},
 {'content': 'You just had a conversation. With whom did you have this conversation?',
  'role': 'user',
  'name': 'Evaluator'},
 {'content': 'I had a conversation with Peter Schmidt from Germany.',
  'role': 'user',
  'name': 'Farhad Abbasi'},
 {'content': 'What was the main topic of this conversation?',
  'role': 'user',
  'name': 'Evaluator'},
 {'content': 'The main topic of the conversation was the rental price for a spacious one-bedroom apartment in the city center.',
  'role': 'user',
  'name': 'Farhad Abbasi'},
 {'content': 'Did you agree on a final price? Answer with "Yes" or "No".',
  'role': 'user',
  'name': 'Evaluator'},
 {'content': 'No, we did not agree on a final price during the conversation.',
  'role': 'user',
  'name': 'Farhad Abbasi'},
 {'content': 'If yes what was the final price? Tell me the exac

In [134]:
answer_name = re.findall(
            r"Emilia Müller|Max Müller|Maryam Abbasi|Farhad Abbasi|Yi-Nuo|Haoyu Wang?",
            conversation_history[1]["content"]
        )[0]
print(answer_name)

answer_agreement = re.findall(
            r"Yes|No?",
            conversation_history[7]["content"]
        )[0]
print(answer_agreement)

# is not converted to float because double is sometimes with . and other times with ,
answer_price = "Null" if len(re.findall( 
            r'[-+]?\d*\,*\d+|[-+]?\d*\.*\d+|[0-9]+',
            conversation_history[9]["content"]
        )) == 0 else re.findall(
            r'[-+]?\d*\,*\d+|[-+]?\d*\.*\d+|[0-9]+',
            conversation_history[9]["content"])[0]
print(answer_price) 

answer_satisfaction = "Null" if len(re.findall(
            r'[0-9]+|\d+\.\d+',
            conversation_history[11]["content"],
        )) == 0 else float(re.findall(
            r'[0-9]+|\d+\.\d+',
            conversation_history[11]["content"],
        )[0])
print(answer_satisfaction)

Farhad Abbasi
No
1,300
5.0


In [135]:
# Empty dataframe which is filled with interview answers
columns = ["answer_name", "answer_agreement", "answer_price", "answer_satisfaction", "city", 
          "model_landlord", "model_renter", "renter_name"]
interviews_results = pd.DataFrame(columns=columns)
display(interviews_results)


,answer_name,answer_agreement,answer_price,answer_satisfaction,city,model_landlord,model_renter,renter_name


In [136]:
if os.path.isdir(full_path):
    for name in os.listdir(full_path):
        # experimental_path = full_path.split(os.path.sep)[1] # extract the experiment path
        name_path = os.path.join(full_path, name)
        for experiment_id in os.listdir(name_path):  # should work (hopefully)
            experiment_id_path = os.path.join(name_path, experiment_id)
            for interview in os.listdir(experiment_id_path):
                interview_path = os.path.join(experiment_id_path, interview)
                for file in os.listdir(interview_path):
                    file_path = os.path.join(interview_path, file)
                    print("File path:", file_path)
                    if os.path.isfile(file_path):
                        print("File found:", file_path)
                        path_parts = file_path.split(os.path.sep)
                        eval_result_sub_path = os.path.join(
                        path_parts[1], path_parts[2], path_parts[3]
                        )
                        
                        experiment_info = re.findall(
                            r"bagel-dpo-34b-v0.2|Yi-34B-Chat|Magdeburg|Duisburg|München|Emilia Müller|Max Müller|Maryam Abbasi|Farhad Abbasi|Yi-Nuo|Haoyu Wang?",
                            name,
                            )
                        model_landlord = experiment_info[0]  # extract the llm used for the renter
                        renter_name = experiment_info[1]
                        model_renter = experiment_info[2]  # extract the llm used for the landlord
                        city = experiment_info[3]
                        interview_partner = interview
                        conversation_history = json.load(open(file_path))
                        answer_name = "Null" if len(re.findall(
                                    r"Emilia Müller|Max Müller|Maryam Abbasi|Farhad Abbasi|Yi-Nuo|Haoyu Wang|Peter Schmidt?",
                                    conversation_history[1]["content"]
                                    )) == 0 else re.findall(
                                    r"Emilia Müller|Max Müller|Maryam Abbasi|Farhad Abbasi|Yi-Nuo|Haoyu Wang|Peter Schmidt?",
                                    conversation_history[1]["content"]
                                    )[0]

                        answer_agreement = "Null" if len(re.findall(
                                            r"Yes|No?",
                                            conversation_history[7]["content"]
                                            )) == 0 else re.findall(
                                            r"Yes|No?",
                                            conversation_history[7]["content"]
                                            )[0]

                        # is not converted to float because double is sometimes with . and other times with ,
                        answer_price = "Null" if len(re.findall( 
                                            r'[-+]?\d*\,*\d+|[-+]?\d*\.*\d+|[0-9]+',
                                            conversation_history[9]["content"]
                                            )) == 0 else re.findall(
                                                r'[-+]?\d*\,*\d+|[-+]?\d*\.*\d+|[0-9]+',
                                                conversation_history[9]["content"])[0]

                        string_satisfaction = conversation_history[11]["content"].replace("1 to 10","")
                        answer_satisfaction = "Null" if len(re.findall(
                                            r'[0-9]+|\d+\.\d+',
                                            string_satisfaction,
                                            )) == 0 else float(re.findall(
                                               r'[0-9]+|\d+\.\d+',
                                                string_satisfaction,
                                            )[0])
                        current_interview = pd.DataFrame([[experiment_id, interview_partner, answer_name, answer_agreement, answer_price, 
                                                           answer_satisfaction, city, model_landlord, 
                                                           model_renter, renter_name]],
                                                         columns = ["experiment_id","interview_partner","answer_name","answer_agreement", 
                                                                    "answer_price", "answer_satisfaction", 
                                                                    "city", "model_landlord", "model_renter", "renter_name"]) 
                        interviews_results = pd.concat([interviews_results, current_interview], ignore_index = True)

                    else:
                        print("File not found:", file_path)
else:
    print("Directory not found:", full_path)

File path: Yi-34B-Chat-Yi-34B-Chat-interview-results-02182024\landlord-Yi-34B-Chat-Emilia Müller from Germany-Yi-34B-Chat-Duisburg-20240207\08572bc2-b5ec-4c3b-ac26-5d6b647c2429\landlord\conversation.json
File found: Yi-34B-Chat-Yi-34B-Chat-interview-results-02182024\landlord-Yi-34B-Chat-Emilia Müller from Germany-Yi-34B-Chat-Duisburg-20240207\08572bc2-b5ec-4c3b-ac26-5d6b647c2429\landlord\conversation.json
File path: Yi-34B-Chat-Yi-34B-Chat-interview-results-02182024\landlord-Yi-34B-Chat-Emilia Müller from Germany-Yi-34B-Chat-Duisburg-20240207\08572bc2-b5ec-4c3b-ac26-5d6b647c2429\renter\conversation.json
File found: Yi-34B-Chat-Yi-34B-Chat-interview-results-02182024\landlord-Yi-34B-Chat-Emilia Müller from Germany-Yi-34B-Chat-Duisburg-20240207\08572bc2-b5ec-4c3b-ac26-5d6b647c2429\renter\conversation.json
File path: Yi-34B-Chat-Yi-34B-Chat-interview-results-02182024\landlord-Yi-34B-Chat-Emilia Müller from Germany-Yi-34B-Chat-Duisburg-20240207\0ec17f2f-98a8-4ebd-aab2-11f5178dd7e2\landlord\

In [137]:
display(interviews_results)

,answer_name,answer_agreement,answer_price,answer_satisfaction,city,model_landlord,model_renter,renter_name,experiment_id,interview_partner
0,Peter Schmidt,Yes,825,Null,Duisburg,Yi-34B-Chat,Yi-34B-Chat,Emilia Müller,08572bc2-b5ec-4c3b-ac26-5d6b647c2429,landlord
1,Emilia Müller,Yes,825,Null,Duisburg,Yi-34B-Chat,Yi-34B-Chat,Emilia Müller,08572bc2-b5ec-4c3b-ac26-5d6b647c2429,renter
2,Emilia Müller,Yes,780,Null,Duisburg,Yi-34B-Chat,Yi-34B-Chat,Emilia Müller,0ec17f2f-98a8-4ebd-aab2-11f5178dd7e2,landlord
3,Emilia Müller,Yes,780,Null,Duisburg,Yi-34B-Chat,Yi-34B-Chat,Emilia Müller,0ec17f2f-98a8-4ebd-aab2-11f5178dd7e2,renter
4,Peter Schmidt,Yes,775,8.0,Duisburg,Yi-34B-Chat,Yi-34B-Chat,Emilia Müller,1ca6ad24-e82f-436a-9380-c70c2443bafb,landlord
...,...,...,...,...,...,...,...,...,...,...
803,Yi-Nuo,Yes,1100,Null,München,Yi-34B-Chat,Yi-34B-Chat,Yi-Nuo,f30ee210-9311-4b87-a929-e5da0bb1bb08,renter
804,Yi-Nuo,Yes,850,8.0,München,Yi-34B-Chat,Yi-34B-Chat,Yi-Nuo,f57f49e0-8a9f-4977-968d-448a5b6f9c71,landlord
805,Yi-Nuo,Yes,850,8.0,München,Yi-34B-Chat,Yi-34B-Chat,Yi-Nuo,f57f49e0-8a9f-4977-968d-448a5b6f9c71,renter
806,Yi-Nuo,No,Null,Null,München,Yi-34B-Chat,Yi-34B-Chat,Yi-Nuo,f59976fc-dd78-4126-b7c5-d003b20e2d19,landlord


In [138]:
output_path = "parsing\output-190224"
interviews_results.to_csv(os.path.join(output_path,results_path+ ".csv"),  encoding='utf-8-sig')